In [1]:
from fraction_class import cFraction


class cMetrics(list):

    def __init__(self, M:list=[], fraction=False):
        if fraction:
            for r in range(len(M)):
                for c in range(len(M[0])):
                    if not(isinstance(M[r][c], cFraction)):
                        M[r][c] = cFraction(M[r][c], 1)
        super().__init__(M) # call list constructor
        self.__updateNoOfRowAndCol() # update NR (no of rows) and NC (no of cols)
        
        
    def __str__(self):
        """
        print list of list in Metrix Form
        """
        NR = self.NR
        NC = self.NC
        string = "[" # outer open bracket
        for row in range(NR):
            string += "[" #  inner open bracket
            for col in range(NC):
                string += str(self[row][col])
                if (col != NC-1): # if entry is not at last of row
                    string += "\t"
            string += "]" # inner closing bracket
            if (row != NR-1): # if row is not the last row of metrics
                string += "\n "
        string += "]" # outer closing brakcet
        return string

    def __eq__(self, M2):
        if (self.NR != M2.NR) or (self.NC != M2.NC): 
            return False
        for r in range(self.NR):
            for c in range(self.NC):
                if (self[r][c] != M2[r][c]):
                    return False
        return True
    
    def __add__(self, M2):
        if (self.NR != M2.NR) or (self.NC != M2.NC):
            raise ValueError("for addition both metrics must be of same size")
        Result = cMetrics() # Empty Metrics
        for r in range(self.NR):
            row = [] # empty row
            for c in range(self.NC):
                row.append(self[r][c] + M2[r][c])
            Result.append(row) # append row to metrics
        return Result

    def __sub__(self, M2):
        R = M2 * -1 # changing sign of second metrics and apply addition process is same as subtraction
        return self + R

    def __getSpecifcEntriesofProduct(self, M2, i, j):
        """
        get specific entry of product Metrics (M1*M2) at row i col j
        """
        if (self.NC != M2.NR): # if matrix product rule does not satisfy
            raise ValueError("column of metrics 1 must be equal to row of metrics 2")
        answer = 0 # entry at row i and col j in product Metrics
        # Multiply corresponding element of row i of M1 and col j of M2 and add up all results
        for k in range(self.NC):
            answer = answer + (self[i][k] * M2[k][j])
        return answer

    def __getProductMetrics(self, M2, NR, NC, row=None, col=None):
        """
        get product Metrics (self * M2)\n
        here:\n
        NR is number of rows of product Metrics\n
        NC is number of columns of product Metrics\n
        row must be given when you want to get specific row of product Metrics\n
        col must be given when you want to get specific column of product Metrics\n
        """
        R = cMetrics()
        for r in range(NR):
            temp = []
            for c in range(NC):
                if (row == None) and (col == None): # if row and column is None get entries of all location
                    answer = self.__getSpecifcEntriesofProduct(M2, r, c)
                elif (row != None) and (col == None): # if row is given, get entries of that row for all columns c
                    answer = self.__getSpecifcEntriesofProduct(M2, row, c)
                elif (row == None) and (col != None): # if column is given, get entries of that col for all row r
                    answer = self.__getSpecifcEntriesofProduct(M2, r, col)
                temp.append(answer)
            R.append(temp)
        return R
    
    def product(self, M2, i=None, j=None, row = None, col = None):
        """
        find product of self and M2\n
        here:\n
        'r' and 'c' is given if you want to get specific entry of product metrics at row r and column c\n
        'row' is given if you want to get specific row of product metrics\n
        'col' is given if you want to get specific column of product metrics\n
        """
        if self.NC != M2.NR: # check for metrix product rule
            raise ValueError("column of metrics 1 must be equal to row of metrics 2")
        
        # handle arguments
        if (i != None) and (j == None): # if r is given for specific entry but c is not given
            raise ValueError("provide column number too for specific entry")
        if (i == None) and (j != None): # if c is given for specific entry but r is not given
            raise ValueError("provide row number too for specific entry")
        if ((row != None) and (col != None)): # if row and col both given at same time
            raise ValueError("You dont get specific row and specifc column at a time")
        if ((i != None)or(j!=None)) and ((row != None)or(col!=None)): # if r or c is given and row or col is given
            raise ValueError("You dont get specific entry and specific row or column at same time")

        if (i == None) and (j == None) and (row == None) and (col == None):
            # if you want to get all entries of product Metrics
            return self.__getProductMetrics(M2, NR=self.NR, NC=M2.NC)
        if (i != None) and (j != None):
            # if you want to get specific entry of product metrics and 'r' and 'c' is given
            return self.__getSpecifcEntriesofProduct(M2, i, j)
        if (row != None):
            # if you want to get specific 'row' of product metrics. 
            # Here output is a row vector, so NR (number of rows of product metrics) is 1
            return self.__getProductMetrics(M2, NR=1, NC=M2.NC, row=row)
        if (col != None):
            # if you want to get specific 'col' of product metrics. 
            # Here output is a col vector, so NC (number of column of product metrics) is 1
            return self.__getProductMetrics(M2, NR=self.NR, NC=1, col=col)



    def __updateNoOfRowAndCol(self):
        """
        update NR and NC attribute of Metrix
        """
        self.NR = len(self) # Number of rows
        if self.NR != 0 and isinstance(self[0], list): # if (list is not empty) and (list contain list)
            self.NC = len(self[0]) # length of inner list
        else:
            self.NC = 0 # otherwise list a row vector

    def append(self, __object) -> None:
        super().append(__object)
        self.__updateNoOfRowAndCol()

    def insert(self, __index, __object) -> None:
        super().insert(__index, __object)
        self.__updateNoOfRowAndCol()

    def copy(self):
        """
        make copy of metrix
        """
        R = cMetrics()
        for r in range(self.NR):
            row = []
            for c in range(self.NC):
                row.append(self[r][c])
            R.append(row)
        return R


    def __mul__(self, k):
        """
        multipy metrix by scalar 'k'
        """
        R = cMetrics()
        for r in range(self.NR):
            row = []
            for c in range(self.NC):
                row.append(self[r][c] * k)
            R.append(row)
        return R
    
    def __truediv__(self, k):
        """
        divide metrix by scalar 'k'
        """
        R = cMetrics()
        for r in range(self.NR):
            row = []
            for c in range(self.NC):
                row.append(self[r][c] / k)
            R.append(row)
        return R
    
    def __makeOne(self, row, pivotElement):
        """
        Make 1 to pivot element.\n
        (Divide metrix row 'row' by pivot element)
        """
        R = self.copy()
        for col in range(R.NC):
            R[row][col]= R[row][col] / pivotElement
        return R

    def __makeZero(self, pr:int, pc:int, upperEntries=True, lowerEntries=True):
        """
        make zero to below and above numbers of pivot element at pivot column 'pc'\n
        if you don't want to make zero to entries above pivot row 'pr', change argument upperEntries to False\n
        if you don't want to make zero to entries below pivot row 'pr'., change argument lowerEtnries to False\n
        """
        if not(upperEntries) and not(lowerEntries):
            raise ValueError("atleast one argument is true; upperTriangular, lowerTriangular")

        
        R = self.copy()
        for r in range(R.NR):
            # loop all row except pivot row
            if r == pr:
                continue
            
            # if you don't want to make zero to entries above pr.
            if not(upperEntries) and r<pr:
                continue
            
            # if you don't want to make zero to entries below pr.
            if not(lowerEntries) and r>pr:
                continue
            # number in pivot column which is either below or above the pivot element 
            # which we want to be zero
            valueToBeZero = R[r][pc]
            # for make zero we have to loop through all column to apply operation
            for c in range(R.NC):
                tempRow = R[pr][c] * valueToBeZero
                R[r][c] = R[r][c] - tempRow
            # eg.  R1   = R1     - 5*R3
            # 5 = value to be zeor
            # R3 = pivot row
            # R1 is the row which contain 5 at pivot column
        return R

    def interchangeTwoRow(self, pos1:int, pos2:int):
        """
        Interchange two row of metrix which are at position 'pos1' and 'pos2'
        """
        # Create copy of metrix to prevent changing in original metrix
        R = self.copy()
        temp = R[pos1] 
        R[pos1] = R[pos2]
        R[pos2] = temp
        return R

    def interchangeTwoColumn(self, pos1:int, pos2:int):
        """
        Interchange two columns of metrix which are at positions 'pos1' and 'pos2'
        """
        R = self.copy()
        for r in range(R.NR):
            temp = R[r][pos1]
            R[r][pos1] = R[r][pos2]
            R[r][pos2] = temp
        return R

    def getReducedRowEcholonForm(self):
        """
        Reduced metrix to row echolon form
        """
        R = self.copy()
        pr = 0
        for pc in range(R.NC):
            # 1.Locate the left most column that does not consist entirely of zeros
            for r in range(pr,R.NR):
                if (R[r][pc] != 0):
                    break
            # if the column does not contain non zero number, continue (move to the next column)
            if (r == R.NR-1) and (R[r][pc] == 0):
                continue
            # 2.Interchange top row 'pr' with row 'r' to bring non zero entry (at row r) to top row (pr)
            if (r != pr):
                R = R.interchangeTwoRow(r, pr)
            pivotElement = R[pr][pc]
            # 3. make pivot element to 1
            R = R.__makeOne(pr, pivotElement)
            # 4. Add sutiable multiples of top row 'pr' to rows below and above so that 
            #    all entries below and above leading 1 is zero 
            R = R.__makeZero(pr, pc)
            # cover top row and begin again with step 1 applied to the submetrix that remains
            pr += 1
            # if pivot row number is greater than the number of rows of oirginal metrix which is happened 
            # due to number of column is greater than number of row in original metrix
            #     and
            # if previous submetix is only contain last row of original metrix then stop processing.
            if (pr > R.NR-1):
                break
        return R

    def transpose(self):
        """
        Interchanging Rows and Columns of Metrix
        if A is (m x n) metrix then transpose of A will be (n x m) metrix
        """
        R = cMetrics() # create empty metrix
        # transpose(Mij) = Mji
        for r in range(self.NC):
            row = []
            for c in range(self.NR):
                # append column entries of original metrix to row of transpose metrix
                row.append(self[c][r])
            R.append(row)
        return R

    def trace(self):
        """
        calculate the trace of scalar metrix\n
        if A is a square metrix then the trace of A is sum of the entries on the main diagonal of A\n
        if A is not a square metrix then trace of metrix is undefined
        """
        if self.NR != self.NC:
            raise ValueError("Trace is undefined")
        answer = 0
        for i in range(self.NR):
            answer = answer + self[i][i]
        return answer
    
    def determinant(self):
        """
        Determinant only exist for Square Metrix\n
        We use Row Reduction method to find deteminant
        """
        # check for determinant condition
        if self.NR != self.NC:
            raise ValueError("Determinat only exist for square metrix")
        # Make copy of original
        R = self.copy()
        # consant will be the deteminant of Metrix
        constant = 1
        for i in range(R.NR):
            pivot = R[i][i]

            # if pivot is zero
            if (pivot == 0):
                # Look through the pivot's right entries for any non-zero entry.
                for c in range(i+1, R.NC):
                    if R[i][c] != 0:
                        break
                # if last entry is zero or there is no non zero entry to left of pivot element, then deteminant is zero
                if (i+1 >= R.NC) or ((c == R.NC-1) and (R[i][c] == 0)):
                    return 0
                # otherwise interchange the column where you find non zero entry, from the pivot column.
                else:
                    R = R.interchangeTwoColumn(i, c)
                    constant = constant * -1
                    # update pivot element
                    pivot = R[i][i]

            # common number from pivot row to make pivot element 1
            constant =  pivot * constant
            R = R.__makeOne(i, pivot)

            # make zero 
            R = R.__makeZero(i, i, upperEntries=False)
        return constant

    def __deleteRowColumn(self, row, col):
        """
        Delete given row and column from matrix
        """
        # make copy of original metrix
        R = self.copy()
        # delete row but R is now a list object not cMetric object
        R = R[:row] + R[row+1:]

        # delete column
        for r in range(len(R)):
            R[r] = R[r][:col] + R[r][col+1:]

        return cMetrics(R)
        

    def minor(self, i, j):
        """
        calculate the minor entry at ith row and jth column
        """
        # minor is only define for square matrix
        if (self.NR != self.NC):
            raise ValueError("A is not a square metrix")

        # delete ith row and jth column
        subMetrix = self.__deleteRowColumn(i, j)
        
        # if subMetrix contain only a number
        if (subMetrix.NR == 1) and (subMetrix.NC == 1):
            return subMetrix[0][0]
        
        # return determinant of the sub metrix
        return subMetrix.determinant()

    
    def cofactor(self, i, j):
        """
        calculate the factor at ith row and jth column
        """

        # factor is only define for square metrix
        if (self.NR != self.NC):
            raise ValueError("A is not a square metrix")
        
        # return (-1)^(i+j) * minor at ith row and jth column
        return  self.minor(i, j) * (-1)**(i+j)

    def cofactorMatrix(self):
        """
        return the cofactor metrix of original metrix
        """

        # cofactor metrix is only define for square metrix
        if (self.NR != self.NC):
            raise ValueError("A is not a square metrix")

        # create empty metrix
        R = cMetrics()
        # loop through all entries
        for r in range(self.NR):
            row = []
            for c in range(self.NC):
                # calculate cofactor at rth row and cth column
                cij = self.cofactor(r, c)
                # add cofactor entry to row
                row.append(cij)
            # add row to metrix
            R.append(row)
        return R

    def adjoint(self):
        """
        return the adjoint metrix
        """
        # adjoint metrix is transpose of cofactor metrix
        return self.cofactorMatrix().transpose()


    def inverse(self):
        """
        return inverse metrix
        """
        # formula = adjoint of metrix / determinant of metrix
        return self.adjoint() / self.determinant()
        

    def applySimplexMethod(self):
        """
        Simplex Method is usually perform on metrix which contains a methamtical model of a business problem\n
        After performing simplex method, the answer is business problem found at last column of metrix
        """
        R = self.copy()

        # multipy top row with -1
        for c in range(R.NC):
            R[0][c] = R[0][c] * -1

        minimum = min(R[0])
        # we apply this process unitll the minimum of top row is greater than 0
        while (minimum < 0):
            # pc-> pivot column: column number which contain most negative number at top row
            pc = R[0].index(minimum)
            # find pivot row; pivot row is the row which 
            # have smallest ratio of (number of row at last column / number of row in pivot column)
            pr = 1 # let the row after top row is pivot row
            smallest_ratio = R[1][-1] / R[1][pc] # let the row after top row have smallest ratio
            # loop through all rows 
            for r in range(1, R.NR):
                ratio = R[r][-1] / R[r][pc] # ratio of rth Row
                # if roatio of rth Row is smaller than samallest ratio
                if ratio < smallest_ratio:
                    # update smallest ratio
                    smallest_ratio = ratio
                    # update pivot row 'pr' to the row r
                    pr = r
            # take out pivot element
            pe = R[pr][pc]

            R.makeOne(pr, pe)
            R.makeZero(pr, pc)
            minimum = min(R[0])


In [21]:
import os, pickle
from metric_class import cMetrics

class VectorSpaceModel():
    
    def wordIndex(self, txt:str, filepath):
        self.docNames.append(filepath)
        words = txt.split()
        row = [0 for _ in self.UWL]
        for index, word in enumerate(words):
            if (word == "") or (word == " "):
                continue
            word = word.lower()            
            for sw in list("!@#$%^&*()_+-={}[]\|;:'""',<.>/?"):
                if sw in word:
                    word = word.replace(sw, "")
            if word not in self.UWL:
                self.UWL.append(word)
                row.append(0)
                for ind, prevRow in enumerate(self.M):
                    self.M[ind] = prevRow + [0]
            row[self.UWL.index(word)] = words.count(word)
        self.M.append(row)

    def scan(self, path):
        try:
            for i in os.scandir(path):
                if (i.is_file()):
                    name = i.name
                    if name.endswith(".txt"):
                        file = open(i.path)
                        txt = file.read()
                        file.close()
                        self.wordIndex(txt, i.path)
                else:
                    self.scan(i)
        except:
            pass

    def createVSM(self, dirPath):
        self.UWL = cMetrics()
        self.M = cMetrics()
        self.docNames = cMetrics()
        self.scan(dirPath)
        self.calculateTermAndDocumentFreq()
        self.applyDocumentFrequency()
        self.applyTermFrequency()


    def writeVSM(self):
        self.writeVectorSpaceToFile()
        self.writeUniqueWordListToFile()
        self.writeDocListToFile()
        self.writeDocFreqToFile()
        self.writeTermFreqToFile()

    def readVSM(self):
        self.UWL = cMetrics()
        self.M = cMetrics()
        self.docNames = cMetrics()
        self.readDocListFromFile()
        self.readUniqueWordListFromFile()
        self.readVectorSpaceFromFile()
        self.readDocFreqFromFile()
        self.readTermFreqFromFile()


    def calculateTermAndDocumentFreq(self):
        self.termFreq = cMetrics()
        for r in self.M:
            self.termFreq.append(sum(r))
            
        self.docFreq = cMetrics()
        for c in range(self.UWL.NR):
            ans = 0
            for r in range(self.M.NR):
                ans += self.M[r][c]
            self.docFreq.append(ans)

    def applyTermFrequency(self):
        for r in range(self.M.NR):
            for c in range(self.UWL.NR):
                if self.termFreq[r] != 0:
                    self.M[r][c] = self.M[r][c] / self.termFreq[r]
        

    def applyDocumentFrequency(self):
        for c in range(self.UWL.NR):
            for r in range(self.M.NR):
                if self.docFreq[c] != 0:
                    self.M[r][c] = self.M[r][c] / self.docFreq[c]

    def writeVectorSpaceToFile(self, path="vsm.pickle"):
        file = open(path, "wb")
        pickle.dump(self.M, file)
        file.close()
    
    def readVectorSpaceFromFile(self, path="vsm.pickle"):
        file = open(path, "rb")
        self.M = pickle.load(file)
        file.close()

    def readUniqueWordListFromFile(self, path="uwl.pickle"):
        file = open(path, "rb")
        self.UWL = pickle.load(file)
        file.close()
        

    def writeUniqueWordListToFile(self, path="uwl.pickle"):
        file = open(path, "wb")
        pickle.dump(self.UWL, file)
        file.close()

    def writeDocListToFile(self, path="doclist.pickle"):
        file = open(path, "wb")
        pickle.dump(self.docNames, file)
        file.close()
    
    def readDocListFromFile(self, path="doclist.pickle"):
        file = open(path, "rb")
        self.docNames = pickle.load(file)
        file.close()
        
    def writeDocFreqToFile(self, path="docfreq.pickle"):
        file = open(path, "wb")
        pickle.dump(self.docFreq, file)
        file.close()
    
    def readDocFreqFromFile(self, path="docfreq.pickle"):
        file = open(path, "rb")
        self.docFreq = pickle.load(file)
        file.close()
        
    def writeTermFreqToFile(self, path="termfreq.pickle"):
        file = open(path, "wb")
        pickle.dump(self.termFreq, file)
        file.close()
    
    def readTermFreqFromFile(self, path="termfreq.pickle"):
        file = open(path, "rb")
        self.termFreq = pickle.load(file)
        file.close()
        


    def createQueryVector(self, query):
        self.queryVector = [[0] for _ in self.UWL]
        self.queryVector = cMetrics(self.queryVector)
        wordsInQuery = query.split(" ")
        for word in wordsInQuery:
            if word in self.UWL:
                self.queryVector[self.UWL.index(word)][0] = 1

    def calculateResult(self):
        self.Result = cMetrics()
        for r in range(self.M.NR):
            ans = 0
            for k in range(self.M.NC):
                ans += self.M[r][k] * self.queryVector[k][0] * self.termFreq[r] * self.docFreq[k]
            self.Result.append([int(ans)])

    def getFileNamesFromResult(self):
        r = zip(*sorted(zip(self.Result, self.docNames), reverse=True))
        self.sortedQueryResult, self.sortedDocNames = r
        for ind, queryDocResult in enumerate(self.sortedQueryResult):
            if queryDocResult[0] == 0.0:
                break
                
        self.sortedDocNames = self.sortedDocNames[:ind]
        self.sortedQueryResult = self.sortedQueryResult[:ind]
        
    def writeResultToHtml(self):
        html = "<!DOCTYPE html><head><title>Result</title></head><body><ol>"
        if len(self.sortedDocNames) == 0:
            html += "No result found"
        for r in range(len(self.sortedDocNames)):
            print(self.sortedDocNames[r], self.sortedQueryResult[r][0])
            html += "<li>"
            filename = self.sortedDocNames[r].split("\\")[-1]
            html += f"<a href='file:\\{self.sortedDocNames[r]}'>{filename}</a> {self.sortedQueryResult[r][0]}</li>"
        html += "</ol></body></html>"
        file = open("result.html","w")
        file.write(html)
        file.close()
        
    def find(self, query):
        self.createQueryVector(query)
        self.calculateResult()
        self.getFileNamesFromResult()
        self.writeResultToHtml()



In [22]:
vsm = VectorSpaceModel()
# # only one time
# vsm.createVSM("C:\\Users\\huzai\\OneDrive\\Documents")
# vsm.writeVSM()

vsm.readVSM()


In [28]:
vsm.find("v106")

In [27]:
vsm.UWL[190]

'v106'

In [29]:
for r in range(vsm.M.NR):
    print(int(vsm.M[r][190]))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [12]:
r

560

In [14]:
vsm.M[r][237]

0

In [15]:
vsm.M.NR

561